**Modules**

In [1]:
import pickle

from configparser import ConfigParser
from mysql.connector import MySQLConnection,Error

**Importing the book_genres_lookup preprocessed data**

In [2]:
with open("../../../../../RecommendationEngine/DataPreparation/Books_Genre_Mapping/book_genres_lookup.pkl", 'rb') as f:
    data_list = pickle.load(f)

In [3]:
data_list[:10]

[(1, 10),
 (1, 9),
 (1, 2),
 (2, 7),
 (2, 10),
 (2, 9),
 (2, 2),
 (3, 6),
 (3, 13),
 (3, 7)]

In [4]:
len(data_list)

7753974

**Checking datatype**

In [5]:
print(data_list[0][0], type(data_list[0][0]))
print(data_list[0][1], type(data_list[0][1]))

1 <class 'int'>
10 <class 'numpy.int64'>


**Updating datatype**

In [6]:
for i,data in enumerate(data_list):
    data_list[i] = (data[0], int(data[1]))

**Crosscheck the datatypes**

In [7]:
print(data_list[0][0], type(data_list[0][0]))
print(data_list[0][1], type(data_list[0][1]))

1 <class 'int'>
10 <class 'int'>


In [8]:
data_list[:10]

[(1, 10),
 (1, 9),
 (1, 2),
 (2, 7),
 (2, 10),
 (2, 9),
 (2, 2),
 (3, 6),
 (3, 13),
 (3, 7)]

**Function to call the proc**

In [9]:
def create_book_genres_lookup(data,cs,proc_name="sp_create_book_genres_lookup",initial=[0,1],end=[0]):
    
    procs_args_complete = []

    procs_args_complete.extend(initial)
    procs_args_complete.extend(data)
    procs_args_complete.extend(end)

    upload_details = cs.callproc(procname=proc_name, args=procs_args_complete)

    return upload_details

**Config reading and parsing**

In [10]:
# reading database credentials from config.ini file

def read(filename='config.ini',section='mysql'):
    parser=ConfigParser()
    parser.read(filename)
    
    db={}
    
    if parser.has_section(section):
        items=parser.items(section)
        for item in items:
            db[item[0]]=item[1]
    else:
        raise Exception(f'{section} not found in file {filename}')
    return db 


print(read(filename="config.ini",section="mysql"))

{'host': '127.0.0.1', 'port': '99966', 'database': 'books', 'user': 'root', 'password': 'Milla123!'}


**Connecting with MySQL/MariaDB database server and getting the connection and cursor object**

In [11]:
def connect(creds):
    con=None
    try:
        print('Connecting to MySQL database...')
        con=MySQLConnection(**creds)
        
        
        if con.is_connected():
            print('Connection established')
            cus = con.cursor(buffered=True)
        else:
            print('Connection failed')
            
    except Error as e:
        print(e)
    finally:
        return con,cus
    
cn,cs=connect(creds=read(filename="config.ini",section="mysql"))

Connecting to MySQL database...
Connection established


### Uploading the Complete Data into the Database

In [12]:
for i, data in enumerate(data_list,start=1):
    create_book_genres_lookup(data=data,cs=cs,proc_name="sp_create_book_genres_lookup",initial=[0,1],end=[0])

    if ((i%100000) == 0):
        cn.commit()

cn.commit()